# BIL564 Derin Öğrenme Projesi – GAN ile Veri Artırımı

## Proje Hedefi:
Bu notebook'ta GAN (Generative Adversarial Network) mimarisi kullanılarak, IQ-OTH/NCCD akciğer kanseri görüntü veri seti üzerinden **sentetik görüntü üretimi** gerçekleştirilmiştir. GAN ile üretilen bu görüntüler, azınlık sınıfların temsilini güçlendirmek ve CNN modelinin sınıflandırma performansını artırmak amacıyla kullanılmaktadır.

## Uygulanan Yöntemler:
- **GAN Eğitimi**: Her sınıf (normal, benign, malignant) için ayrı GAN mimarileri oluşturulmuştur.
- **Generator & Discriminator**: Giriş olarak rastgele latent vektör alıp sahte görüntüler üretir; discriminator ise bu görüntülerin gerçekliğini değerlendirir.
- **Discriminator Tabanlı Filtreleme**: Yalnızca `güven skoru > 0.95` olan sentetik görüntüler eğitimde kullanılmak üzere seçilmiştir.
- **Görselleştirme**: Gerçek ve sentetik görüntüler görsel olarak karşılaştırılmış, kalite değerlendirmesi yapılmıştır.

## Ana Aşamalar:
1. Kaggle üzerinden veri setinin indirilmesi
2. Görüntülerin yeniden boyutlandırılması ve TensorFlow dataset'e dönüştürülmesi
3. GAN mimarilerinin tanımlanması ve kayıp fonksiyonlarının oluşturulması
4. Her sınıf için GAN eğitimi ve sentetik veri üretimi
5. Yüksek kaliteli sahte görüntülerin `.png` formatında kaydedilmesi
6. Gerçek ve GAN görüntülerinin görselleştirilmesi

## Kaggle Üzerinden Veri Setinin İndirilmesi

Bu hücrede Kaggle API kullanılarak IQ-OTHNCCD akciğer kanseri veri seti indirilmektedir.  
- `kaggle.json` dosyası yüklenerek API erişimi sağlanır.  
- Dataset, çalışma dizinine `.zip` formatında indirilir.  


In [ ]:
from google.colab import files
import os, zipfile

# 1. kaggle.json dosyasını yükle
print("Lütfen kaggle.json dosyanı yükle:")
uploaded = files.upload()

# 2. Kaggle API ayarları
os.makedirs("/root/.kaggle", exist_ok=True)
with open("/root/.kaggle/kaggle.json", "wb") as f:
    f.write(uploaded["kaggle.json"])
os.chmod("/root/.kaggle/kaggle.json", 600)

# 3. Dataset'i indir (senin verdiğin path)
!kaggle datasets download -d hamdallak/the-iqothnccd-lung-cancer-dataset

# 4. Zip dosyasını çıkar
with zipfile.ZipFile("the-iqothnccd-lung-cancer-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("lung_dataset")

## Kaggle'dan İndirilen ZIP Dosyasının Açılması

Bu hücrede, Kaggle üzerinden indirilen `The IQ-OTHNCCD lung cancer dataset.zip` dosyası çıkarılmaktadır.  
- ZIP dosyasının yolu manuel olarak belirtilmiştir.  
- İçerikler `/content/lung_dataset` klasörüne açılır.  

> Bu adım, verileri eğitimde kullanmak üzere hazır hale getirir.


## Gerekli Kütüphanelerin Yüklenmesi

Bu hücrede proje boyunca kullanılacak tüm kütüphaneler içe aktarılmaktadır.  
- `TensorFlow` ve `Keras`: GAN, CNN modeli ve veri işleme işlemleri için kullanılır.  
- `NumPy`, `Matplotlib`, `PIL`, `os`: Görüntü işleme, dosya yönetimi ve görselleştirme için kullanılır.  
- `scikit-learn`: Veri setini eğitim/test olarak ayırmak için kullanılır.  
- Kod tekrarı ve karmaşayı önlemek amacıyla gereksiz ve yinelenen importlar temizlenmiştir.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
import os
import imghdr
import random
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Rescaling
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import numpy as np
import numpy as np
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
import os
import random
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from PIL import Image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt




## Veri Setinin Yüklenmesi ve Ön İşleme

Bu hücrede IQ-OTHNCCD akciğer kanseri veri seti TensorFlow `image_dataset_from_directory` fonksiyonu ile yüklenmektedir.

### Temel Adımlar:
- **Yol Tanımlama:** Görsellerin bulunduğu dizin belirtilmiştir.
- **Parametreler:** Görseller 128x128 boyutuna yeniden boyutlandırılmış, batch boyutu 32 olarak ayarlanmıştır.
- **Normalization:** Görseller [-1, 1] aralığına normalize edilmiştir (`Rescaling(1./127.5, offset=-1)`).
- **Train/Test Ayrımı:** %80 eğitim, %20 doğrulama olarak ayrılmıştır.

### Performans Optimizasyonu:
- `cache()`, `prefetch()` ve `shuffle()` işlemleri eğitim sürecini hızlandırmak için kullanılmıştır.

### Sınıf Bilgisi:
- Veri setinde 3 sınıf mevcuttur: **Benign**, **Malignant** ve **Normal**.
- Bu sınıflar etiket-dizin eşleşmeleri ile bir sözlükte tutulmaktadır.

### Sınıf Bazlı Ayrım:
- Eğitim verisi içerisinden her bir sınıf (benign, malignant, normal) için ayrı dataset’ler filtrelenmiştir.


In [ ]:
dataset_path = Path("/content/lung_dataset/The IQ-OTHNCCD lung cancer dataset")

image_size = (128, 128)
batch_size = 32
seed = 123

normalization_layer = Rescaling(1./127.5, offset=-1)
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32)
    image = normalization_layer(image)
    return image, label

raw_train_dataset = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size
)
raw_val_dataset = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size
)

class_names = raw_train_dataset.class_names
print("Sınıf İsimleri:", class_names)

train_dataset = (
    raw_train_dataset
    .map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(1000)
    .cache()
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

val_dataset = (
    raw_val_dataset
    .map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

label_to_index = {name: i for i, name in enumerate(class_names)}
print("Etiket eşleşmeleri:", label_to_index)


def filter_by_class(dataset, class_index):
    return dataset.filter(lambda img, label: tf.reduce_any(tf.equal(label, class_index)))

train_dataset_benign = filter_by_class(train_dataset, label_to_index["Bengin cases"])
train_dataset_malignant = filter_by_class(train_dataset, label_to_index["Malignant cases"])
train_dataset_normal = filter_by_class(train_dataset, label_to_index["Normal cases"])


## GAN Mimarisi: Generator ve Discriminator

### Generator
- Giriş: 100 boyutlu rastgele bir latent vektör.
- 4 katmanlı transpoz konvolüsyonel yapı ile 128x128 boyutunda RGB görüntüler üretir.
- Aktivasyon: `tanh` ile [-1, 1] aralığında normalize edilmiş çıktı verir.

### Discriminator
- Giriş: 128x128 boyutunda bir görüntü (gerçek veya sahte).
- 4 katmanlı konvolüsyonel yapı ve sonrasında `Dense(1)` ile gerçeklik olasılığı tahmini yapar.
- Aktivasyon: `sigmoid`.


In [ ]:
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(16 * 16 * 256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((16, 16, 256)),

        layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', use_bias=False),  # (16->32)
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', use_bias=False),   # (32->64)
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        layers.Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same', activation='tanh')  # (64->128)
    ])
    return model

def build_discriminator():
    model = tf.keras.Sequential([
        # Input layer accepts 128x128 RGB images
        layers.InputLayer(input_shape=(128, 128, 3)),

        # First convolution: 128x128 -> 64x64
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.4),

        # Second convolution: 64x64 -> 32x32
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.4),

        # Third convolution: 32x32 -> 16x16
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.4),

        # Fourth convolution: 16x16 -> 8x8
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.5),

        # Flatten and output a single probability
        layers.Flatten(),  # Output shape: (None, 8*8*512) = (None, 32768)
        layers.Dense(1, activation='sigmoid')
    ])

    return model

# Normal GAN
gen_normal = build_generator()
disc_normal = build_discriminator()

# Benign GAN
gen_benign = build_generator()
disc_benign = build_discriminator()

# Malignant GAN
gen_malignant = build_generator()
disc_malignant = build_discriminator()

## GAN Kayıp Fonksiyonları (Loss Functions)

### Discriminator Loss
- Gerçek görüntüler için `label=1`, sahte görüntüler için `label=0` hedeflenir.
- Gerçek ve sahte görüntüler için ayrı ayrı `BinaryCrossentropy` hesaplanır.
- Toplam kayıp: `real_loss + fake_loss`.

### Generator Loss
- Amaç: Üretilen sahte görüntülerin `Discriminator` tarafından **gerçek** olarak sınıflandırılmasını sağlamaktır.
- Bu nedenle, tüm `fake_output`’lar için `label=1` olarak verilir.

> `BinaryCrossentropy(from_logits=False)` kullanılmıştır çünkü çıktı aktivasyon fonksiyonu `sigmoid` ile normalize edilmiştir.


In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def discriminator_loss(real_output, fake_output):
    real_output = tf.cast(real_output, tf.float32)
    fake_output = tf.cast(fake_output, tf.float32)

    real_labels = tf.ones_like(real_output, dtype=tf.float32)
    fake_labels = tf.zeros_like(fake_output, dtype=tf.float32)

    real_loss = cross_entropy(real_labels, real_output)
    fake_loss = cross_entropy(fake_labels, fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    fake_output = tf.cast(fake_output, tf.float32)
    labels = tf.ones_like(fake_output, dtype=tf.float32)
    return cross_entropy(labels, fake_output)


### Optimizers

- Bu hücrede, hem Generator hem de Discriminator için Adam optimizasyon algoritması tanımlanmıştır. Bu optimizasyon, ağların öğrenme sürecini hızlandırmak ve stabil hale getirmek için kullanılır.


In [ ]:
generator_optimizer = tf.keras.optimizers.AdamW(learning_rate=0.0001, weight_decay=1e-4, beta_1=0.5, beta_2=0.999)
discriminator_optimizer = tf.keras.optimizers.AdamW(learning_rate=0.0002, weight_decay=1e-4, beta_1=0.5, beta_2=0.999)


## GAN Eğitim Süreci

### `train_step()`:
- Bir batch veri alınarak:
  - Latent vektör ile sahte görüntüler üretilir.
  - Gerçek ve sahte görüntüler `Discriminator` üzerinden geçer.
  - Her iki modelin loss değerleri hesaplanır ve geri yayılım (backpropagation) yapılır.

---

### `train_gan()`:
- Verilen epoch sayısı kadar `train_step()` çalıştırılır.
- Her `vis_interval` epoch’ta bir sentetik görüntüler oluşturularak görselleştirilir.
- Eğitim sonunda:
  - Generator ve Discriminator modelleri `.h5` formatında kaydedilir.
  - Final sentetik örnekler gösterilir.

---

### `generate_and_plot_images()`:
- Generator ile örnek latent vektörlerden görseller üretilir.
- Görseller bir grid şeklinde görselleştirilir (`matplotlib` ile).

> Bu yapı, ileride entegre edeceğimiz **Streamlit uygulamasında** da kullanılacak olan temel GAN eğitim mantığını oluşturur.


In [ ]:

def train_step(data, generator, discriminator, generator_optimizer, discriminator_optimizer):
    images, _ = data
    noise = tf.random.normal([tf.shape(images)[0], 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        dis_loss = discriminator_loss(real_output, fake_output)

    generator_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(dis_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))

    return gen_loss, dis_loss


def train_gan(dataset, epochs, generator, discriminator, generator_optimizer, discriminator_optimizer, test_images, model_name, vis_interval=5):
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")

        total_gen_loss = 0.0
        total_dis_loss = 0.0
        num_batches = 0

        for image_batch in dataset:
            gen_loss, dis_loss = train_step(image_batch, generator, discriminator, generator_optimizer, discriminator_optimizer)
            total_gen_loss += gen_loss
            total_dis_loss += dis_loss
            num_batches += 1

        avg_gen_loss = total_gen_loss / num_batches
        avg_dis_loss = total_dis_loss / num_batches

        print(f"Generator Loss: {avg_gen_loss:.4f}, Discriminator Loss: {avg_dis_loss:.4f}")

        if (epoch + 1) % vis_interval == 0 or epoch == 0:
            generate_and_plot_images(generator, test_images, title=f"{model_name} - Epoch {epoch + 1}")

    print(f"\n✅ Eğitim tamamlandı: {model_name}")
    generate_and_plot_images(generator, test_images, title=f"{model_name} - Final")
    generator.save(f"{model_name}_generator.h5")
    discriminator.save(f"{model_name}_discriminator.h5")
    print(f"💾 Kaydedildi: {model_name}_generator.h5, {model_name}_discriminator.h5")


def generate_and_plot_images(generator, test_images, title="GAN Görüntüleri"):
    generated_images = generator(test_images, training=False)
    fig = plt.figure(figsize=(8, 8))
    grid_size = int(np.sqrt(generated_images.shape[0]))

    for i in range(generated_images.shape[0]):
        plt.subplot(grid_size, grid_size, i + 1)
        plt.imshow((generated_images[i].numpy() + 1) / 2)
        plt.axis('off')

    plt.suptitle(title, fontsize=16)
    plt.show()


## GAN Eğitiminin Başlatılması

### Sabit Test Gürültüsü
- Eğitim sırasında görsellerin dönemsel olarak üretilebilmesi için 16 adet sabit latent vektör (`test_images`) oluşturulmuştur.


###  `run_training()` Fonksiyonu
- Belirtilen veri kümesi için:
  - Yeni bir **Generator** ve **Discriminator** oluşturulur.
  - `AdamW` optimizasyon algoritması ile her iki model eğitilir.
  - Eğitim `train_gan()` fonksiyonu ile yürütülür.
  - Klavye ile eğitim durdurulursa modeller geçici olarak kaydedilir (`checkpoint`).

---

### Çoklu GAN Eğitimi
- Her sınıf (Normal, Benign, Malignant) için ayrı GAN modelleri eğitilebilir.
- Şu anda yalnızca **Malignant** sınıfı için eğitim başlatılmıştır:

```python
run_training(train_dataset_malignant, "gan_malignant")


In [ ]:
# Sabit test gürültüleri
noise_dimensions = 100
num_test_images = 16
np.random.seed(123)
tf.random.set_seed(123)
test_images = tf.random.normal([num_test_images, noise_dimensions])

# GPU için ayar (opsiyonel)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Eğitim fonksiyonu çağrısı
def run_training(dataset, model_name):
    generator = build_generator()
    discriminator = build_discriminator()

    generator_optimizer = tf.keras.optimizers.AdamW(learning_rate=0.00015, weight_decay=1e-4, beta_1=0.5, beta_2=0.999)
    discriminator_optimizer = tf.keras.optimizers.AdamW(learning_rate=0.0001, weight_decay=1e-4, beta_1=0.5, beta_2=0.999)

    try:
        train_gan(
            dataset=dataset,
            epochs=3000,
            generator=generator,
            discriminator=discriminator,
            generator_optimizer=generator_optimizer,
            discriminator_optimizer=discriminator_optimizer,
            test_images=test_images,
            model_name=model_name
        )
    except KeyboardInterrupt:
        print(f"\n Eğitim durduruldu: {model_name}")
        generator.save(f"{model_name}_generator_interrupted.h5")
        discriminator.save(f"{model_name}_discriminator_interrupted.h5")
        print("💾 Checkpoint kaydedildi.")


# GAN eğitimini başlat
run_training(train_dataset_normal, "gan_normal")
run_training(train_dataset_benign, "gan_benign")




#İleri aşamalarda buraya gerek olmadığını analiz ettim

#run_training(train_dataset_malignant, "gan_malignant")



## GAN ile Sentetik Görüntü Üretimi ve Kaydedilmesi (Tüm Sınıflar)

Bu hücre, daha önce eğitilmiş GAN modelleri (`gan_benign.h5`, `gan_malignant.h5`, `gan_normal.h5`) kullanılarak **her bir sınıf için 400 adet sentetik görüntü** üretmek amacıyla çalıştırılır.

### İşlem Adımları:
1. **Model Yükleme:** İlgili sınıfa ait GAN modeli `.h5` uzantısıyla yüklenir.
2. **Latent Boyutu:** Modelin giriş şekli (`latent_dim`) otomatik olarak algılanır.
3. **Latent Vektör Oluşturma:** `z ∼ N(0, 1)` dağılımından istenilen sayıda örnek üretilir.
4. **Görüntü Üretimi:** Generator model kullanılarak sentetik görüntüler elde edilir.
5. **Görüntü Kaydı:**
   - Görüntüler normalize edilip (0–255 aralığına getirilip), `.png` olarak uygun klasöre kaydedilir.
   - Örn: `generated_images_benign/`, `generated_images_malignant/`, `generated_images_normal/`

> Bu işlem her sınıf için ayrı ayrı çalıştırılmalıdır. Üretilen görüntüler, CNN modeli için veri setini genişletmede (augmentation) kullanılacaktır.


In [ ]:
generator = load_model("gan_benign_generator.h5")


input_shape = generator.input_shape
latent_dim = input_shape[1]

print(f"[INFO] Detected latent dimension: {latent_dim}")


output_dir = "generated_benign_normal"
os.makedirs(output_dir, exist_ok=True)


num_images = 1000
latent_vectors = np.random.normal(0, 1, (num_images, latent_dim))


generated_images = generator.predict(latent_vectors, verbose=1)


for i, img in enumerate(generated_images):

    img = np.clip(img, 0, 1)


    img = (img * 255).astype(np.uint8)


    if img.shape[-1] == 1:
        img = img.squeeze(-1)


    plt.imsave(f"{output_dir}/image_{i+1:03d}.png", img, cmap='gray' if len(img.shape) == 2 else None)


## Gerçek ve GAN Görüntülerinin Karşılaştırmalı Görselleştirmesi
### Görselleştirilen Sınıflar:
- Normal cases
- Benign cases

### Adımlar:
1. Her sınıf için:
   - `num_samples` kadar **gerçek** ve **sentetik** görüntü rastgele seçilir.
2. Görseller bir subplot grid içerisinde yan yana yerleştirilir:
   - Sol sütunlar: Gerçek görüntüler.
   - Sağ sütunlar: GAN tarafından üretilmiş sentetik görüntüler.

> Bu karşılaştırma, GAN modellerinin görsel kalitesini değerlendirmek ve üretimlerinin gerçek görüntülere ne kadar yakın olduğunu gözlemlemek için önemlidir.


In [ ]:
classes = ["Normal cases", "Bengin cases"]
num_samples = 5

real_base_path = "/content/lung_dataset/The IQ-OTHNCCD lung cancer dataset"
gan_base_paths = {
    "Normal cases": "generated_images_normal",
    "Bengin cases": "generated_images_benign"
}

fig, axs = plt.subplots(len(classes), num_samples * 2, figsize=(num_samples * 2.5, len(classes) * 3))

for row_idx, class_name in enumerate(classes):

    real_path = os.path.join(real_base_path, class_name)
    gan_path = gan_base_paths[class_name]

    real_images = random.sample(os.listdir(real_path), num_samples)
    gan_images = random.sample(os.listdir(gan_path), num_samples)

    for i in range(num_samples):

        real_img = Image.open(os.path.join(real_path, real_images[i]))
        axs[row_idx, i].imshow(real_img, cmap='gray' if real_img.mode == 'L' else None)
        axs[row_idx, i].axis('off')
        axs[row_idx, i].set_title(f"{class_name} - Real", fontsize=8)


        gan_img = Image.open(os.path.join(gan_path, gan_images[i]))
        axs[row_idx, num_samples + i].imshow(gan_img, cmap='gray' if gan_img.mode == 'L' else None)
        axs[row_idx, num_samples + i].axis('off')
        axs[row_idx, num_samples + i].set_title(f"{class_name} - GAN", fontsize=8)

plt.tight_layout()
plt.show()
